In [ ]:
import numpy as np
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

import ocelot
from fel import SASE, random_beam, random_geometry, N_ELEMENTS

hidden_rng = np.random.RandomState(1111)

In [ ]:
class Optimizer(object):
    def __init__(self, x0):
        """
        Optimizer should accept initial configuration x0
        """
        pass

    def reset(self, ):
        """
        Auxilary method --- resets internal state of the optimiser.
        """
        raise NotImplementedError()
        
    def ask(self,):
        """
        Returns next configuration to probe.
        """
        raise NotImplementedError()
    
    def tell(self, x, f):
        """
        Callback method:
        `x` - configuration returned by `ask()` method (possibly clipped to satisfy bounds),
        `f` - value of the objective function in the point `x`.
        """
        raise NotImplementedError()

In [ ]:
class SimulatedAnnealing(Optimizer):
    def __init__(self, x0, T0=0.2, T_final=0.0, n_steps=100, scale=0.1, seed=1122):
        self.x0 = x0
        self.rng = np.random.RandomState(seed=seed)
        
        self.x = x0
        self.f = None
        
        self.T0 = T0
        self.T = T0
        self.T_final = T_final
        self.n_steps = n_steps
        self.i = 0
        
        self.momentum = 0*x0
        
        self.scale = scale
        
        super(SimulatedAnnealing, self).__init__(x0)
    
    def ask(self, ):
        return self.x + self.rng.normal(size=self.x.shape, scale=self.scale)
    
    def tell(self, x, f):
        
        if self.i<self.n_steps and self.n_steps>0:
            self.T = self.T0 - (self.T0 - self.T_final) * self.i /self.n_steps
            self.i += 1
        
        if self.f is None:
            self.f = f
            self.x = x

        elif self.f < f:
            if self.f-f > np.log(self.rng.uniform(0,1)+1e-20)*self.T:
                self.f = f
                self.x = x     
            else:
                pass

        else:
            # self.momentum += (x-self.x)*(self.f-f)**2 # does not work very well...
            self.f = f
            self.x = x
    
    def reset(self,):
        self.x = self.x0
        self.f = None

In [ ]:
class GradientDescent(Optimizer):
    def __init__(self, x0, learning_rate=0.5, seed=1122):
        self.x0 = x0
        self.x = x0
        # self.rng = np.random.RandomState(seed=seed)
        
        self.learning_rate = learning_rate
        self.grad = np.zeros(shape=x0.shape)
        
        self.i = 0
        self.h = 0.05
        self.f0 = 0
        self.f = None
        
        super(GradientDescent, self).__init__(x0)
    
    def ask(self, ):
        
        if self.i == 0:     
            return self.x
        elif self.i==11:
            self.i = 0
            self.x -= self.learning_rate * self.grad
            return self.x 
        else:
            hi = 0*self.x
            hi[self.i-1] = self.h
            return self.x + hi
    
    def tell(self, x, f):
        
        self.f = f
        
        if self.i == 0:
            self.f0 = self.f
        elif self.i>0 and self.i<11:
            self.grad[self.i-1] = (self.f - self.f0)/self.h          
        self.i += 1
    
    def reset(self,):
        self.x = self.x0
        self.f = None

In [ ]:
class RandomSearch(Optimizer):
    def __init__(self, x0, seed=1122):
        self.x0 = x0
        self.rng = np.random.RandomState(seed=seed)
        
        super(RandomSearch, self).__init__(x0)
    
    def ask(self, ):
        return self.rng.uniform(-1, 1, size=self.x0.shape)
    
    def tell(self, x, f):
        pass
    
    def reset(self,):
        pass

In [ ]:
class GreedySearch(Optimizer):
    def __init__(self, x0, scale=1e-2, seed=1122):
        self.x0 = x0
        self.rng = np.random.RandomState(seed=seed)
        self.x = x0
        self.f = None
        
        self.scale = scale
        
        super(GreedySearch, self).__init__(x0)
    
    def ask(self, ):
        return self.x + self.rng.normal(size=self.x.shape, scale=self.scale)
    
    def tell(self, x, f):
        if self.f is None:
            self.f = f
            self.x = x

        elif self.f < f:
            pass

        else:
            self.f = f
            self.x = x
    
    def reset(self,):
        self.x = self.x0
        self.f = None

In [ ]:
sase = SASE(random_beam(hidden_rng), random_geometry(hidden_rng))
epsilon = 1e-12

objective = lambda x: np.log(1e-3) - np.log(sase.rho_int(x) + epsilon)

bounds = np.stack([
    -2 * np.ones(sase.ndim()),
    2 * np.ones(sase.ndim())
], axis=1)

x0 = np.zeros(sase.ndim())

In [ ]:
%%timeit
objective(np.random.uniform(-1, 1, size=sase.ndim()))

In [ ]:
def eval_optimization(
    optimiser_class, f, x0, bounds,
    moving_cost=1, measuring_cost=1, budget=128,
    progress=None, *args, **kwargs
):
    if progress is None:
        progress = lambda x: x
    
    optimiser = optimiser_class(x0, *args, **kwargs)    
    
    current_x = np.zeros_like(x0)
    
    history_x = list()
    history_f = list()
    
    max_iterations = int(np.floor(budget / measuring_cost))
    
    for _ in progress(range(max_iterations)):
        x = optimiser.ask()
        x = np.array(x, dtype=np.float64)        
        assert x.shape == x0.shape
        
        x = np.clip(x, bounds[:, 0], bounds[:, 1])

        distance = np.max(np.abs(current_x - x))
        
        cost = moving_cost * distance + measuring_cost
        
        if budget < cost:
            break
        
        budget -= cost
        value = f(x)
        current_x = x
        
        optimiser.tell(x, value)
        
        history_x.append(np.copy(x))
        history_f.append(np.copy(value))
    
    return np.array(history_x), np.array(history_f)

In [ ]:
from scipy import optimize

solution = optimize.minimize(
    objective,
    x0=x0,
    method='Powell',
    options=dict(maxfev=128),
    bounds=bounds
)

In [ ]:
solution.message

In [ ]:
print(solution.fun)
print(solution.x)

In [ ]:
def cummin(fs):
    result = np.zeros_like(fs)
    
    result[0] = fs[0]
    
    for i in range(1, fs.shape[0]):
        if result[i - 1] < fs[i]:
            result[i] = result[i - 1]
        else:
            result[i] = fs[i]
    
    return result

In [ ]:
xs, fs = eval_optimization(
    GreedySearch,
    objective,
    x0=x0,
    bounds=bounds,
    progress=tqdm
)

np.min(fs)

In [ ]:
plt.plot(cummin(fs))
plt.scatter(np.arange(fs.shape[0]), fs, alpha=0.25)
plt.xlabel('iteration')

In [ ]:
xs, fs = eval_optimization(
    RandomSearch,
    objective,
    x0=x0,
    bounds=bounds,
    progress=tqdm
)

np.min(fs)

In [ ]:
plt.plot(cummin(fs))
plt.scatter(np.arange(fs.shape[0]), fs, alpha=0.25)
plt.xlabel('iteration')

In [ ]:
# Gradient descent, quite expensive in 10 dimensions...
xs, fs = eval_optimization(
    GradientDescent,
    objective,
    x0=x0,
    budget=128,
    bounds=bounds,
    progress=tqdm,
)
np.min(fs)

In [ ]:
# Our best result so far...
xs, fs = eval_optimization(
    SimulatedAnnealing,
    objective,
    x0=x0,
    budget=128,
    bounds=bounds,
    progress=tqdm,
    n_steps=100,
)
np.min(fs)

In [ ]:
plt.plot(cummin(fs))
plt.scatter(np.arange(fs.shape[0]), fs, alpha=0.25)
plt.xlabel('iteration')

In [ ]:
# Takes 40 min to run
# Results are below
results = {}
for T0 in [0.025, 0.05,0.1,0.2, 0.3]:
    for scale in [0.1,0.2,0.3,0.4]:
        xs, fs = eval_optimization(
            SimulatedAnnealing,
            objective,
            x0=x0,
            bounds=bounds,
            budget=128,
            progress=tqdm,
            T0=T0,
            scale=scale,
        )
        results[(T0,scale)] = np.min(fs)

In [ ]:
# results found at the previous step
results = {(0.025, 0.1): 1.3154044364721127,
 (0.025, 0.2): 1.1757902170882772,
 (0.025, 0.3): 1.2140989988120285,
 (0.025, 0.4): 1.1021724705975267,
 (0.05, 0.1): 1.0680840365131088,
 (0.05, 0.2): 1.0562616676943977,
 (0.05, 0.3): 1.3358007446730689,
 (0.05, 0.4): 1.1021724705975267,
 (0.1, 0.1): 1.3213544529982943,
 (0.1, 0.2): 1.2475638784992764,
 (0.1, 0.3): 1.1256058788500773,
 (0.1, 0.4): 1.0453218299679161,
 (0.2, 0.1): 0.950515009912424,
 (0.2, 0.2): 1.3265010537112918,
 (0.2, 0.3): 1.005551951459343,
 (0.2, 0.4): 1.1430089224559339,
 (0.3, 0.1): 1.4415174467767677,
 (0.3, 0.2): 1.390773292624976,
 (0.3, 0.3): 1.1713928452618596,
 (0.3, 0.4): 0.9378377359564425}

In [ ]:
min(results.values())

In [ ]:
xs